In [1]:
import networkx as nx
import matplotlib.pyplot as plt 
import pandas as pd
import seaborn as sns
import logging
import numpy as np
import geopandas as gpd
from geopy.geocoders import Nominatim
from shapely.wkt import loads
from scipy.spatial.distance import cdist

import plotly.graph_objects as go

# Let's brag about digital twins ;)

import sgp_dt as dt


In [3]:
# to play with the lib in real time
%load_ext autoreload 
%autoreload 2

# Checking what we have in the network

In [6]:
# For reference only, the geodataframe with all path
fname = 'data/s4.zip'
SLC = gpd.read_file(fname)
SLC

,osm_id,code,fclass,name,ref,oneway,maxspeed,layer,bridge,tunnel,Length,Lamps20m,CCTV20mRE,Trees20m,geometry
0,1,5115,tertiary,Keng Lee Road,None,F,50,0,F,F,100.099656,4,0,74,"LINESTRING (103.84379 1.31370, 103.84468 1.31383)"
1,2,5115,tertiary,Keng Lee Road,None,F,50,0,F,F,38.075346,4,0,25,"LINESTRING (103.84468 1.31383, 103.84495 1.313..."
2,3,5153,footway,None,None,B,0,0,F,F,99.549892,1,0,8,"LINESTRING (103.84083 1.31463, 103.84115 1.31379)"
3,4,5153,footway,None,None,B,0,0,F,F,39.468936,2,0,12,"LINESTRING (103.84115 1.31379, 103.84126 1.313..."
4,5,5153,footway,None,None,B,0,0,F,F,8.235514,2,0,17,"LINESTRING (103.84128 1.31346, 103.84132 1.31340)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51416,51417,5153,footway,None,None,B,0,0,F,F,35.922454,0,0,4,"LINESTRING (103.84721 1.29759, 103.84738 1.297..."
51417,51418,5153,footway,None,None,B,0,0,F,F,6.686514,0,0,2,"LINESTRING (103.84721 1.29759, 103.84721 1.29753)"
51418,51419,5153,footway,None,None,B,0,0,F,F,0.040900,0,0,2,"LINESTRING (103.84721 1.29753, 103.84721 1.29753)"
51419,51420,5153,footway,None,None,B,0,0,F,F,3.673968,0,0,3,"LINESTRING (103.84721 1.29753, 103.84720 1.29750)"


#### With some stats

In [8]:
stats = SLC.groupby('fclass')['Length'].agg(['sum','count']).astype(int).reset_index()
stats.columns = ["network","length (m)","number of segments"]
stats

,network,length (m),number of segments
0,cycleway,30173,781
1,footway,502033,20457
2,living_street,2058,87
3,path,10495,206
4,pedestrian,67048,3761
5,residential,175763,7926
6,secondary,52127,2074
7,secondary_link,1673,114
8,service,305331,13214
9,steps,8882,702


# Starting the streamlit

In [10]:
def loadShp(path):
    # Reading the overall network
    G= nx.readwrite.nx_shp.read_shp(path)
    # Support to network identification
    df_nodes = pd.read_csv("data/SG_nodes.txt", index_col=0)
    # Some cleaning necessary for csv loadup
    dfNodes = pd.read_csv("data/dfNodes.csv.zip")
    dfNodes.Pos = dfNodes.Pos.apply(lambda x: eval(x))
    dfEdges = pd.read_csv("data/dfEdges.csv.zip") 
    dfEdges.geometry = dfEdges.geometry.apply(lambda x: loads(x.replace('\'', '')))
    # Additional layers  
    gTrees = gpd.read_file('data/sTrees.zip') 
    gLamps = gpd.read_file('data/sLamps.zip') 
    gPark = gpd.read_file('data/sParks.zip') 
    gCCTV = gpd.read_file('data/sCCTV.zip') 
    return G, df_nodes, dfNodes, dfEdges, gTrees, gLamps, gPark, gCCTV

# This call should be cached
G, df_nodes, dfNodes, dfEdges, gTrees, gLamps, gPark, gCCTV = loadShp("data/s4/SingaporeLampsCCTVTrees.shp")

In [11]:
security   = 1 
cctv_perf  = 5.0
lamps_perf = 5.0
trees_perf = 5.0
#Different values for different types for paths
footway     = 2.0
residential = 1.5 
steps       = 2.0

UseDummyPoints = True

if UseDummyPoints:
    start,end = dt.getStartEnd("nop", "end", df_nodes, dummy= UseDummyPoints)
else:
    start,end = dt.getStartEnd(start_point, end_point, df_nodes, dummy= UseDummyPoints)

In [22]:
def get_weighted_graphTrees(G,security=1,cctv_perf=1000.0,lamps_perf=1000.0,trees_perf=1000.0):
    """
    Takes a graph G as input and adds the weights
    """
    weighted_G = nx.Graph()
    for data in G.edges(data=True):
        data=data
        coord1 = data[0]
        coord2 = data[1]

        # ZE formula to tweak
        data[2]['weight'] = data[2]['Length']
        data[2]['weight'] = data[2]['weight'] * (1.0 + 0.1*data[2]['CCTV20mRE'] * cctv_perf )
        data[2]['weight'] = data[2]['weight'] * (1.0 + 0.1*data[2]['Lamps20m']  * lamps_perf) 
        data[2]['weight'] = data[2]['weight'] * (1.0 + 0.1*data[2]['Trees20m']  * trees_perf) 

        # It is adapted for pedestrians ?
        relativeEase = 1.0
        if data[2]['fclass'] == "pedestrian":
            relativeEase = 0.3
        elif data[2]['fclass'] == "footway":
            relativeEase = 0.3
        elif data[2]['fclass'] == "steps":
            relativeEase = 0.6 
        
        #data[2]['weight'] = data[2]['weight'] * relativeEase

        weighted_G.add_edge(coord1,coord2,weight=data[2]['weight'])

    return weighted_G

security   = 1.0
cctv_perf  = 1.0
lamps_perf = 1.0
trees_perf = 1.0

weighted_G = get_weighted_graphTrees(G,security=1,cctv_perf=cctv_perf,lamps_perf=lamps_perf,trees_perf=trees_perf)

In [23]:
dt.mapIt(start,end,weighted_G,dfNodes,dfEdges)